In [1]:
# 觀測資料查詢系統CODiS: https://e-service.cwb.gov.tw/HistoryDataQuery/index.jsp

# 466910_鞍部

In [2]:
import pandas as pd
from calendar import monthrange
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup

import numpy as np

In [3]:
year = '2022'
month = '06'

In [4]:
df = pd.DataFrame()

In [5]:
for i in tqdm(range(monthrange(int(year), int(month))[1])):
    url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=466910&\
stname=%25E9%259E%258D%25E9%2583%25A8&datepicker='+year+'-'+month+'-'+f'{i+1:02d}'+'&altitude=837.6m'
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find('tbody').find_all('td')
    l = []
    
    for j in result:
        l.append(j.getText().strip())
    
    temp = np.reshape(l, (24, 17))
    temp = temp.astype('<U10')
    temp = pd.DataFrame(np.reshape(np.insert(temp.T, 0, [year+'-'+month+'-'+f'{i+1:02d}']*24), (18, 24)).T)
    
    df = pd.concat([df, temp])

100%|██████████| 30/30 [38:05<00:00, 76.17s/it]


In [6]:
df.columns = ['date', 'ObsTime', 'StnPres', 'SeaPres', 'Temperature', 'Td dew point',
              'RH', 'WS', 'WD', 'WSGust', 'WDGust', 'Precp', 'PrecpHour', 'SunShine',
              'GloblRad', 'Visb', 'UVI', 'Cloud Amount']
df.head()

,date,ObsTime,StnPres,SeaPres,Temperature,Td dew point,RH,WS,WD,WSGust,WDGust,Precp,PrecpHour,SunShine,GloblRad,Visb,UVI,Cloud Amount
0,2022-06-01,01,918.3,1498.1,21.5,21.5,100,1.2,80,4.7,110,0.0,0.0,...,0.00,...,0,...
1,2022-06-01,02,917.8,1493.4,21.5,21.5,100,2.2,180,6.3,160,0.0,0.0,...,0.00,...,0,...
2,2022-06-01,03,917.4,1489.4,21.4,21.4,100,1.1,190,6.9,140,0.0,0.0,...,0.00,...,0,...
3,2022-06-01,04,917.4,1488.7,20.7,20.7,100,1.6,120,6.7,150,0.0,0.0,...,0.00,...,0,...
4,2022-06-01,05,917.2,1487.2,21.0,21.0,100,2.5,150,8.8,150,0.0,0.0,...,0.00,...,0,...


In [7]:
df.to_excel('氣象觀測站_'+year+month+'.xlsx')